In [ ]:
import os
os.environ["DDE_BACKEND"] = "pytorch"
import deepxde as dde

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
from datetime import datetime
import shutil
from matplotlib.collections import LineCollection
import deepxde as dde
import torch

# =========================================
# MODEL PARAMETERS
# =========================================
# Geometry parameters
L, W, H = 10.0, 1.0, 1.0  # Beam dimensions
total_time = 5.0          # Total simulation time
E = 1e3                   # Young's modulus
nu = 0.47                 # Poisson's ratio
mu = E / (2 * (1 + nu))   # Shear modulus
lambda_ = E * nu / ((1 + nu) * (1 - 2 * nu))  # First Lamé parameter
rho = 0.1                 # Density
kappa = E / (3 * (1 - 2 * nu))  # Bulk modulus

scale_factor = 5.0        # Initial scale factor
mu_effective = mu / scale_factor  # Effective shear modulus
lambda_effective = lambda_ / scale_factor  # Effective Lamé parameter
stiffness_damping = 0.05  # Stiffness damping
mass_damping = 0.05       # Mass damping
gravity_vector = [0.0, 0.0, -9.81]  # Gravity vector
epsilon = 1e-6

In [ ]:
def modify_output(inputs, outputs):
    # Modify outputs with a boundary mask
    x = inputs[:, 0:1]
    mask_xy = torch.sigmoid(x / L)
    mask_z = torch.sigmoid(x / L)   # Different handling for z to promote downward bending
    mask = torch.cat([mask_xy, mask_xy, mask_z], dim=1)
    return outputs * mask

def neo_hookean_pde_stable(x, y):
    global mu_effective, lambda_effective, L, rho, gravity_vector, FORCE_LEVEL, epsilon

    u = y[:, 0:1]  # displacement in x
    v = y[:, 1:2]  # displacement in y
    w = y[:, 2:3]  # displacement in z
    x_coord = x[:, 0:1]
    t = x[:, 3:4]

    load_factor = torch.sigmoid(5.0 * (t - 0.2)) * FORCE_LEVEL

    def grad(var, inputs, idx):
        return torch.autograd.grad(var.sum(), inputs, create_graph=True)[0][:, idx:idx+1]

    du_dx = grad(u, x, 0)
    du_dy = grad(u, x, 1)
    du_dz = grad(u, x, 2)
    dv_dx = grad(v, x, 0)
    dv_dy = grad(v, x, 1)
    dv_dz = grad(v, x, 2)
    dw_dx = grad(w, x, 0)
    dw_dy = grad(w, x, 1)
    dw_dz = grad(w, x, 2)

    F_xx = 1 + du_dx; F_xy = du_dy; F_xz = du_dz
    F_yx = dv_dx; F_yy = 1 + dv_dy; F_yz = dv_dz
    F_zx = dw_dx; F_zy = dw_dy; F_zz = 1 + dw_dz

    J = F_xx * (F_yy * F_zz - F_yz * F_zy) - F_xy * (F_yx * F_zz - F_yz * F_zx) + F_xz * (F_yx * F_zy - F_yy * F_zx)
    J = torch.clamp(J, min=epsilon) + epsilon
    inv_J = 1 / J
    log_J = torch.log(J)
    term = lambda_effective * log_J - mu_effective

    P_xx = mu_effective * F_xx + term * (F_yy * F_zz - F_yz * F_zy) * inv_J
    P_xy = mu_effective * F_xy - term * (F_yx * F_zz - F_yz * F_zx) * inv_J
    P_xz = mu_effective * F_xz + term * (F_yx * F_zy - F_yy * F_zx) * inv_J
    P_yx = mu_effective * F_yx + term * (F_xy * F_zz - F_xz * F_zy) * inv_J
    P_yy = mu_effective * F_yy - term * (F_xx * F_zz - F_xz * F_zx) * inv_J
    P_yz = mu_effective * F_yz + term * (F_xx * F_zy - F_xy * F_zx) * inv_J
    P_zx = mu_effective * F_zx + term * (F_xy * F_yz - F_xz * F_yy) * inv_J
    P_zy = mu_effective * F_zy - term * (F_xx * F_yz - F_xy * F_xz) * inv_J
    P_zz = mu_effective * F_zz + term * (F_xx * F_yy - F_xy * F_yx) * inv_J

    div_P_x = grad(P_xx, x, 0) + grad(P_xy, x, 1) + grad(P_xz, x, 2)
    div_P_y = grad(P_yx, x, 0) + grad(P_yy, x, 1) + grad(P_yz, x, 2)
    div_P_z = grad(P_zx, x, 0) + grad(P_zy, x, 1) + grad(P_zz, x, 2)

    force_magnitude = -5000.0 * load_factor
    force_mask = (x_coord > (L - 0.1)).float()
    force_z = force_magnitude * force_mask
    gravity_z = rho * gravity_vector[2]

    scale = 1.0
    eq_x = div_P_x * scale
    eq_y = div_P_y * scale
    eq_z = (div_P_z + force_z + gravity_z) * scale

    return [eq_x, eq_y, eq_z]

def create_geometry_with_time():
    spatial = dde.geometry.Cuboid(xmin=[0, 0, 0], xmax=[L, W, H])
    time_dom = dde.geometry.TimeDomain(0, total_time)
    return dde.geometry.GeometryXTime(spatial, time_dom)

def boundary_fixed(x, on_boundary):
    return on_boundary and np.isclose(x[0], 0.0, atol=1e-5)

def dirichlet_condition(x):
    return [0, 0, 0]


In [ ]:
def generate_extra_points():
    points = []

    """
    Generates additional training points with adaptive refinement.
    Points are concentrated in regions of interest: near the fixed end,
    along the top and bottom surfaces, and at the free end.
    """

    # Points near the fixed end (x=0)
    for i in range(15):
        x_val = L * (1 - np.exp(-5 * i / 15))
        for y_val in [0, W/2, W]:
            for z_val in [0, H/2, H]:
                for t_val in np.linspace(0, total_time, 5):
                    points.append([x_val, y_val, z_val, t_val])

    # High-density points at the free end
    for x_val in np.linspace(L-1, L, 15):
        for y_val in [0, W/4, W/2, 3*W/4, W]:
            for z_val in [0, H/4, H/2, 3*H/4, H]:
                for t_val in np.linspace(0, total_time, 8):
                    points.append([x_val, y_val, z_val, t_val])

    # Points along the edges
    for x_val in np.linspace(0, L, 20):
        for y_val in [0, W]:
            for z_val in [0, H]:
                for t_val in np.linspace(0, total_time, 5):
                    points.append([x_val, y_val, z_val, t_val])

    # Points through the thickness at critical x locations
    for x_val in [0, L/4, L/2, 3*L/4, L]:
        for z_val in np.linspace(0, H, 8):
            for t_val in np.linspace(0, total_time, 5):
                points.append([x_val, W/2, z_val, t_val])

    # Points at critical time steps for transient behavior
    critical_times = np.concatenate([
        np.linspace(0, 0.1 * total_time, 10),
        np.linspace(0.1 * total_time, total_time, 10)
    ])
    for x_val in [L/4, L/2, 3*L/4, L]:
        for t_val in critical_times:
            points.append([x_val, W/2, H/2, t_val])

    return np.array(points)


In [ ]:
# Callbacks for monitoring and visualization

class EarlyStoppingCallback(dde.callbacks.Callback):
    def __init__(self, patience=2000, min_delta=1e-5, monitor_every=100, min_iterations=1000):
        super().__init__()
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')
        self.wait_count = 0
        self.monitor_every = monitor_every
        self.min_iterations = min_iterations
        self.loss_history = []

    def on_epoch_end(self):
        iter_num = self.model.train_state.iteration
        if iter_num % self.monitor_every != 0 or iter_num < self.min_iterations:
            return

        current_loss = self.model.train_state.loss_train
        if isinstance(current_loss, (list, np.ndarray)):
            current_loss = np.sum(current_loss)
        self.loss_history.append(current_loss)

        if current_loss < self.best_loss - self.min_delta:
            self.best_loss = current_loss
            self.wait_count = 0
            print(f"Iteration {iter_num}: New best loss: {self.best_loss:.8f}")
        else:
            self.wait_count += 1

        if len(self.loss_history) > 5:
            recent_losses = self.loss_history[-5:]
            oscillating = all(recent_losses[i] > recent_losses[i-1] for i in range(1, 5, 2)) and \
                          all(recent_losses[i] < recent_losses[i-1] for i in range(2, 5, 2))
            if oscillating:
                print("Warning: Loss is oscillating. Consider adjusting learning rate or regularization.")

        if self.wait_count >= self.patience // self.monitor_every:
            self.model.stop_training = True
            print(f"\nEarly stopping at iteration {iter_num}")
            print(f"Best loss: {self.best_loss:.8f}")

class VisualizationCallback(dde.callbacks.Callback):
    def __init__(self, interval=500, exaggerate_factors=[1.0, 3.0]):
        super().__init__()
        self.interval = interval
        self.exaggerate_factors = exaggerate_factors
        os.makedirs("results/training_progress", exist_ok=True)

    def on_epoch_end(self):
        iter_num = self.model.train_state.iteration
        if iter_num % self.interval != 0:
            return

        x = np.linspace(0, L, 100)
        y = np.ones_like(x) * (W / 2)
        z = np.ones_like(x) * (H / 2)
        t = np.ones_like(x) * total_time
        points = np.column_stack([x, y, z, t])
        predictions = self.model.predict(points)
        disp_x, disp_y, disp_z = predictions[:, 0], predictions[:, 1], predictions[:, 2]
        strain_x = np.gradient(disp_x, x)
        strain_z = np.gradient(disp_z, x)
        param_text = f"Material: E={E:.1e} Pa, ν={nu:.3f}\nDims: {L}×{W}×{H} m\nForce: {FORCE_LEVEL*3000:.1f} N"

        for factor in self.exaggerate_factors:
            plt.figure(figsize=(12, 8))
            plt.plot(x, np.zeros_like(x), 'k--', label='Original position')
            deformed_x = x + factor * disp_x
            deformed_z = z + factor * disp_z
            plt.plot(deformed_x, deformed_z, 'r-', linewidth=2, label=f'Deformed (x{factor})')
            plt.title(f'Beam Deformation at Iteration {iter_num} (x{factor})')
            plt.xlabel('X position (m)')
            plt.ylabel('Z position (m)')
            plt.grid(True)
            plt.legend()

            max_disp_z = np.min(disp_z)
            max_disp_x = np.max(np.abs(disp_x))
            max_strain_x = np.max(np.abs(strain_x))
            max_strain_z = np.max(np.abs(strain_z))
            plt.figtext(0.02, 0.02,
                        f'{param_text}\n\nMax Z-disp: {max_disp_z:.6f} m\nMax X-disp: {max_disp_x:.6f} m\n'
                        f'Max X-strain: {max_strain_x:.6f}\nMax Z-strain: {max_strain_z:.6f}',
                        bbox=dict(facecolor='white', alpha=0.8))
            plt.savefig(f'results/training_progress/deformation_factor{factor}_iter_{iter_num:05d}.png', dpi=200)
            plt.close()

        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.plot(x, strain_x, 'b-', linewidth=2)
        plt.title('Axial Strain (X)')
        plt.xlabel('X (m)')
        plt.ylabel('Strain')
        plt.grid(True)
        plt.subplot(1, 2, 2)
        plt.plot(x, strain_z, 'r-', linewidth=2)
        plt.title('Vertical Strain (Z)')
        plt.xlabel('X (m)')
        plt.ylabel('Strain')
        plt.grid(True)
        plt.figtext(0.02, 0.02, param_text, bbox=dict(facecolor='white', alpha=0.8))
        plt.tight_layout()
        plt.savefig(f'results/training_progress/strain_distribution_iter_{iter_num:05d}.png', dpi=200)
        plt.close()

        print(f"\nVisualization at iteration {iter_num} saved")
        print(f"Material: E={E:.1e} Pa, ν={nu:.3f}, Force={FORCE_LEVEL*3000:.1f} N")
        print(f"Max Z disp: {max_disp_z:.6f} m, Max X disp: {max_disp_x:.6f} m")
        print(f"Max strains - X: {max_strain_x:.6f}, Z: {max_strain_z:.6f}")




class LossMonitorCallback(dde.callbacks.Callback):
    def __init__(self, monitor_every=100, components_names=None):
        super().__init__()
        self.monitor_every = monitor_every
        self.loss_history = []
        self.components_names = components_names or ['BC_x', 'BC_y', 'BC_z', 'PDE_x', 'PDE_y', 'PDE_z']

    def on_epoch_end(self):
        iter_num = self.model.train_state.iteration
        if iter_num % self.monitor_every != 0:
            return

        try:
            loss_vals = self.model.train_state.loss_train
            if isinstance(loss_vals, np.ndarray):
                total_loss = np.sum(loss_vals)
                self.loss_history.append((iter_num, total_loss, loss_vals))
                print(f"\nIteration {iter_num}, Material: E={E:.1e} Pa, ν={nu:.3f}")
                print(f"Dims: {L}×{W}×{H} m, Force: {FORCE_LEVEL*3000:.1f} N")
                print(f"Total loss: {total_loss:.8f}")
                if len(loss_vals) >= len(self.components_names):
                    loss_dict = {name: loss_vals[i] for i, name in enumerate(self.components_names)}
                    print("Loss components:")
                    max_len = max(len(name) for name in loss_dict)
                    for name, value in loss_dict.items():
                        print(f"  {name.ljust(max_len)}: {value:.8f}")
                    print("\nComponent percentages:")
                    for name, value in loss_dict.items():
                        pct = (value / total_loss) * 100
                        print(f"  {name.ljust(max_len)}: {pct:.2f}%")
                if len(loss_vals) >= 3 and iter_num > 1000:
                    if loss_vals[0] > 100 * loss_vals[2] or loss_vals[2] > 100 * loss_vals[0]:
                        print("\nWarning: Significant imbalance between x and z equation residuals.")
                        print("Consider adjusting loss weights.")
            else:
                print(f"Iteration {iter_num}, Material: E={E:.1e} Pa, ν={nu:.3f}")
                print(f"Dims: {L}×{W}×{H} m, Force: {FORCE_LEVEL*3000:.1f} N")
                print(f"Total loss: {loss_vals:.8f}")
                self.loss_history.append((iter_num, loss_vals, None))

            if iter_num % 1000 == 0 and len(self.loss_history) >= 5:
                recent = self.loss_history[-5:]
                losses = [item[1] for item in recent]
                change = ((losses[-1] - losses[0]) / losses[0]) * 100 if losses[0] > 0 else 0
                print(f"\nLoss trend over last 5 checkpoints: {change:.2f}% change")
                if change > 0:
                    print("Warning: Loss is increasing.")
                elif abs(change) < 0.1:
                    print("Note: Loss improvement has plateaued (< 0.1% change).")
        except Exception as e:
            print(f"Error monitoring losses: {e}")


In [ ]:
class EnhancedVisualizer3DCallback(dde.callbacks.Callback):
    """
    3D visualization callback that generates high-quality deformation plots
    with surfaces colored by strain.
    """
    def __init__(self, interval=500, exaggerate_factors=[1.0, 5.0], L=10.0, W=1.0, H=1.0, total_time=10.0):
        super().__init__()
        self.interval = interval
        self.exaggerate_factors = exaggerate_factors
        self.L = L
        self.W = W
        self.H = H
        self.total_time = total_time
        self.output_dir = "results/3d_visualizations"
        os.makedirs(self.output_dir, exist_ok=True)
        self.count = 0

    def on_epoch_end(self):
        if self.model.train_state.iteration % self.interval != 0:
            return

        iteration = self.model.train_state.iteration
        try:
            # Generate grid for 3D visualization
            nx, ny, nz = 20, 5, 5
            x = np.linspace(0, self.L, nx)
            y = np.linspace(0, self.W, ny)
            z = np.linspace(0, self.H, nz)

            # Centerline points for strain calculation
            x_line = np.linspace(0, self.L, 100)
            y_line = np.ones_like(x_line) * (self.W / 2)
            z_line = np.ones_like(x_line) * (self.H / 2)
            t_line = np.ones_like(x_line) * self.total_time

            line_points = np.column_stack([x_line, y_line, z_line, t_line])
            line_preds = self.model.predict(line_points)
            disp_x_line = line_preds[:, 0]
            disp_z_line = line_preds[:, 2]

            strain_x = np.gradient(disp_x_line, x_line)
            strain_z = np.gradient(disp_z_line, x_line)
            strain_mag = np.sqrt(strain_x**2 + strain_z**2)

            max_disp_x = np.max(np.abs(disp_x_line))
            max_disp_z = np.max(np.abs(disp_z_line))
            min_disp_z = np.min(disp_z_line)
            max_strain = np.max(strain_mag)

            param_text = (f"Material properties:\nE: {E:.1e} Pa, ν: {nu:.3f}\n"
                          f"Dimensions: {L}×{W}×{H} m\n"
                          f"Force: {FORCE_LEVEL*3000:.1f} N")

            for factor in self.exaggerate_factors:
                self._create_3d_surface_plot(x, y, z, factor, strain_mag,
                                             max_disp_x, max_disp_z, min_disp_z, max_strain,
                                             iteration, param_text)
                self._create_side_view(x_line, y_line, z_line, disp_x_line, disp_z_line,
                                       strain_mag, factor, iteration, param_text)

            print(f"\nIteration {iteration} visualization saved")
            print(f"Parameters: E={E:.1e}Pa, ν={nu:.3f}, Force={FORCE_LEVEL*3000:.1f}N")
            print(f"Min Z disp: {min_disp_z:.6f} m, Max Z disp: {max_disp_z:.6f} m, "
                  f"Max X disp: {max_disp_x:.6f} m, Max strain: {max_strain:.6f}")
            self.count += 1

        except Exception as e:
            import traceback
            print(f"Error during visualization: {e}")
            traceback.print_exc()
            print("Continuing training...")

    def _create_3d_surface_plot(self, x, y, z, factor, strain_mag, max_disp_x, max_disp_z, min_disp_z, max_strain, iteration, param_text):
        fig = plt.figure(figsize=(14, 10))
        ax = fig.add_subplot(111, projection='3d')

        def visualize_face(face):
            if face == 'bottom':
                X, Y = np.meshgrid(x, y)
                Z = np.zeros_like(X)
            elif face == 'top':
                X, Y = np.meshgrid(x, y)
                Z = np.ones_like(X) * self.H
            elif face == 'front':
                X, Z = np.meshgrid(x, z)
                Y = np.zeros_like(X)
            elif face == 'back':
                X, Z = np.meshgrid(x, z)
                Y = np.ones_like(X) * self.W
            elif face == 'left':
                Y, Z = np.meshgrid(y, z)
                X = np.zeros_like(Y)
            elif face == 'right':
                Y, Z = np.meshgrid(y, z)
                X = np.ones_like(Y) * self.L
            mesh_orig = (X, Y, Z)
            pts = np.column_stack([X.flatten(), Y.flatten(), Z.flatten(), np.full(X.flatten().shape, self.total_time)])
            predictions = self.model.predict(pts)
            disp_x = predictions[:, 0]
            disp_y = predictions[:, 1]
            disp_z = predictions[:, 2]
            if face in ['bottom', 'top', 'front', 'back']:
                strain_proxy = np.abs(disp_z) / max_disp_z if max_disp_z > 0 else np.zeros_like(disp_z)
            else:
                strain_proxy = np.abs(disp_x) / max_disp_x if max_disp_x > 0 else np.zeros_like(disp_x)
            colors = plt.cm.viridis(strain_proxy)
            deformed_x = pts[:, 0] + factor * disp_x
            deformed_y = pts[:, 1] + factor * disp_y
            deformed_z = pts[:, 2] + factor * disp_z
            if face in ['bottom', 'top']:
                original_shape = X.shape
                X_def = deformed_x.reshape(original_shape)
                Y_def = deformed_y.reshape(original_shape)
                Z_def = deformed_z.reshape(original_shape)
                ax.plot_surface(X, Y, Z, color='lightblue', alpha=0.2, rstride=1, cstride=1, linewidth=0.2)
                ax.plot_surface(X_def, Y_def, Z_def, alpha=0.8, rstride=1, cstride=1, linewidth=0.2,
                                facecolors=colors.reshape(*original_shape, 4))
            else:
                original_shape = X.shape
                X_def = deformed_x.reshape(original_shape)
                Y_def = deformed_y.reshape(original_shape)
                Z_def = deformed_z.reshape(original_shape)
                ax.plot_surface(X, Y, Z, color='lightblue', alpha=0.2, rstride=1, cstride=1, linewidth=0.2)
                ax.plot_surface(X_def, Y_def, Z_def, alpha=0.8, rstride=1, cstride=1, linewidth=0.2,
                                facecolors=colors.reshape(*original_shape, 4))

        for face in ['bottom', 'top', 'front', 'back', 'left', 'right']:
            visualize_face(face)

        ax.set_xlabel('X (m)')
        ax.set_ylabel('Y (m)')
        ax.set_zlabel('Z (m)')
        ax.set_title(f'Beam Deformation (Factor: {factor}x) - Iteration {iteration}')
        ax.view_init(elev=25, azim=30)
        ax.set_box_aspect([self.L, self.W, self.H])
        sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis)
        sm.set_array([])
        cbar = plt.colorbar(sm, ax=ax, shrink=0.6, aspect=20)
        cbar.set_label('Relative Strain')
        from matplotlib.patches import Patch
        legend_elements = [Patch(facecolor='lightblue', alpha=0.4, label='Original'),
                           Patch(facecolor='green', alpha=0.8, label='High strain'),
                           Patch(facecolor='blue', alpha=0.8, label='Low strain')]
        ax.legend(handles=legend_elements, loc='upper right')
        metrics_text = (f"{param_text}\n\nMax displacements:\n  Z: {min_disp_z:.6f} m (downward), "
                        f"X: {max_disp_x:.6f} m\nMax strain: {max_strain:.6f}")
        plt.figtext(0.02, 0.02, metrics_text,
                    bbox=dict(facecolor='white', alpha=0.8, boxstyle='round,pad=0.5'), fontsize=10)
        plt.tight_layout()
        plt.savefig(f"{self.output_dir}/deformation_3d_factor{int(factor)}_iter{iteration:05d}.png", dpi=300)
        plt.close()

    def _create_side_view(self, x_line, y_line, z_line, disp_x_line, disp_z_line, strain_mag, factor, iteration, param_text):
        fig = plt.figure(figsize=(12, 6))
        plt.plot(x_line, z_line, 'k--', linewidth=1.5, label='Original')
        deformed_x = x_line + factor * disp_x_line
        deformed_z = z_line + factor * disp_z_line
        points = np.vstack([deformed_x, deformed_z]).T
        segments = np.array([points[:-1], points[1:]]).transpose(1, 0, 2)
        from matplotlib.collections import LineCollection
        lc = LineCollection(segments, cmap='viridis', linewidth=3)
        lc.set_array(strain_mag[:-1])
        plt.gca().add_collection(lc)
        plt.colorbar(lc, label='Strain Magnitude')
        plt.xlabel('X (m)')
        plt.ylabel('Z (m)')
        plt.title(f'Side View with Strain (Factor: {factor}x) - Iteration {iteration}')
        plt.grid(True)
        plt.legend()
        plt.figtext(0.02, 0.02, param_text,
                    bbox=dict(facecolor='white', alpha=0.8, boxstyle='round,pad=0.5'), fontsize=10)
        plt.axis('equal')
        plt.tight_layout()
        plt.savefig(f"{self.output_dir}/side_view_factor{int(factor)}_iter{iteration:05d}.png", dpi=300)
        plt.close()


In [ ]:
def create_enhanced_animation(model, num_frames=30, exaggerate_factor=3.0):
    """
    Creates an animation of beam deformation over time with strain visualization.

    Args:
        model: Trained PINN model.
        num_frames: Number of frames in the animation.
        exaggerate_factor: Factor to exaggerate deformations.
    """
    output_dir = "results/animation"
    os.makedirs(output_dir, exist_ok=True)

    param_text = (f"Material properties:\nE: {E:.1e} Pa, ν: {nu:.3f}\n"
                  f"Dimensions: {L}×{W}×{H} m\n"
                  f"Force: {FORCE_LEVEL*3000:.1f} N")

    # Create time points (more frames during initial loading)
    initial_times = np.linspace(0, 0.2 * total_time, int(num_frames * 0.4))
    later_times = np.linspace(0.2 * total_time, total_time, int(num_frames * 0.6))
    time_points = np.concatenate([initial_times, later_times])

    x = np.linspace(0, L, 100)
    y_center = W / 2
    z_center = H / 2
    frames = []

    for i, t in enumerate(time_points):
        fig = plt.figure(figsize=(12, 10))

        # Deformation subplot
        ax1 = fig.add_subplot(211)
        points = np.column_stack([x, np.full_like(x, y_center), np.full_like(x, z_center), np.full_like(x, t)])
        predictions = model.predict(points)
        disp_x = predictions[:, 0]
        disp_z = predictions[:, 2]

        strain_x = np.gradient(disp_x, x)
        strain_z = np.gradient(disp_z, x)
        strain_mag = np.sqrt(strain_x**2 + strain_z**2)

        ax1.plot(x, np.zeros_like(x), 'k--', label='Original position')
        deformed_x = x + exaggerate_factor * disp_x
        deformed_z = z_center + exaggerate_factor * disp_z
        pts = np.array([deformed_x, deformed_z]).T
        segments = np.array([pts[:-1], pts[1:]]).transpose(1, 0, 2)

        from matplotlib.collections import LineCollection
        lc = LineCollection(segments, cmap='coolwarm', linewidth=3)
        lc.set_array(strain_mag[:-1])
        ax1.add_collection(lc)
        cbar = plt.colorbar(lc, ax=ax1)
        cbar.set_label('Strain Magnitude')

        ax1.set_xlabel('X position (m)')
        ax1.set_ylabel('Z position (m)')
        ax1.set_title(f'Beam Deformation at t={t:.2f}s (x{exaggerate_factor})')
        ax1.grid(True)
        ax1.set_xlim(-0.5, L + 0.5)
        ax1.set_ylim(-2, 2)
        ax1.legend(loc='upper right')

        # Strain subplot
        ax2 = fig.add_subplot(212)
        ax2.plot(x, strain_x, 'b-', linewidth=2, label='Axial strain')
        ax2.plot(x, strain_z, 'r-', linewidth=2, label='Bending strain')
        ax2.plot(x, strain_mag, 'g-', linewidth=1.5, label='Total strain')
        ax2.set_xlabel('X position (m)')
        ax2.set_ylabel('Strain')
        ax2.set_title(f'Strain Distribution at t={t:.2f}s')
        ax2.grid(True)
        ax2.legend(loc='upper right')

        max_disp_z = np.abs(disp_z).max()
        min_disp_z = np.min(disp_z)
        max_strain = np.max(strain_mag)

        metrics_text = (f"{param_text}\n\nTime: {t:.2f}s\n"
                        f"Max downward disp: {min_disp_z:.6f} m\n"
                        f"Max strain: {max_strain:.6f}")
        plt.figtext(0.02, 0.02, metrics_text,
                    bbox=dict(facecolor='white', alpha=0.8, boxstyle='round,pad=0.5'),
                    fontsize=10)

        plt.tight_layout()
        frame_filename = f"{output_dir}/frame_{i:03d}.png"
        plt.savefig(frame_filename, dpi=200)
        frames.append(frame_filename)
        plt.close()

    # Create animated GIF if PIL is available
    try:
        from PIL import Image
        images = [Image.open(f) for f in frames]
        images[0].save(f"{output_dir}/beam_deformation_animation.gif",
                       save_all=True, append_images=images[1:], duration=100, loop=0)
        print(f"Animation saved to {output_dir}/beam_deformation_animation.gif")
    except ImportError:
        print("PIL not installed. Frames saved as PNG files.")

    # Create MP4 video if moviepy is available
    try:
        import moviepy.editor as mpy
        clip = mpy.ImageSequenceClip(frames, fps=10)
        clip.write_videofile(f"{output_dir}/beam_deformation_animation.mp4", fps=10)
        print(f"Video saved to {output_dir}/beam_deformation_animation.mp4")
    except ImportError:
        print("Moviepy not installed. MP4 video not created.")

    return frames


In [ ]:
def multi_config_training(base_model=None):
    """
    Trains multiple PINN configurations by varying parameters and saving results.
    """
    master_dir = f"multi_config_pinn_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    os.makedirs(master_dir, exist_ok=True)
    print("=" * 80)
    print("MULTI-CONFIGURATION PINN TRAINING")
    print("=" * 80)

    configurations = [
        {
            'name': 'config1_baseline',
            'description': 'Baseline configuration with standard parameters',
            'global_vars': {
                'E': 1e3,
                'nu': 0.47,
                'total_time': 5.0
            },
            'training': {
                'force_levels': [0.05, 0.1, 0.2, 0.3, 0.5, 0.8, 1.0],
                'loss_weights': [1.0, 1.0, 0.1, 1.0e9, 1.0e9, 8.0e9]
            }
        },
        {
            'name': 'config2_material',
            'description': "Modified material properties (higher Young's modulus)",
            'global_vars': {
                'E': 2e3,
                'nu': 0.45,
                'total_time': 5.0
            },
            'training': {
                'force_levels': [0.05, 0.1, 0.2, 0.3, 0.5, 0.8, 1.0],
                'loss_weights': [1.0, 1.0, 0.01, 1.0e9, 1.0e9, 8.0e9]
            }
        },
        {
            'name': 'config3_geometry',
            'description': 'Modified geometry (longer, thinner beam)',
            'global_vars': {
                'L': 12.0,
                'W': 0.8,
                'H': 0.8,
                'E': 1e3,
                'nu': 0.47,
                'total_time': 5.0
            },
            'training': {
                'force_levels': [0.05, 0.1, 0.2, 0.3, 0.5, 0.8, 1.0],
                'loss_weights': [1.0, 1.0, 0.01, 1.0e9, 1.0e9, 8.0e9]
            }
        },
        {
            'name': 'config4_loading',
            'description': 'Modified loading condition (higher force)',
            'global_vars': {
                'E': 1e3,
                'nu': 0.47,
                'total_time': 5.0
            },
            'training': {
                'force_levels': [0.05, 0.1, 0.2, 0.4, 0.6, 0.8, 1.2],
                'loss_weights': [1.0, 1.0, 0.1, 1.0e9, 1.0e9, 8.0e9]
            },
            'pde_params': {
                'force_magnitude': -6000.0
            }
        }
    ]

    with open(os.path.join(master_dir, "configurations_summary.txt"), "w") as f:
        f.write("MULTI-CONFIGURATION PINN BEAM STUDY\n")
        f.write("=================================\n\n")
        f.write(f"Created: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        for i, config in enumerate(configurations):
            f.write(f"Configuration {i+1}: {config['name']}\n")
            f.write("-" * 50 + "\n")
            f.write(f"Description: {config['description']}\n\n")
            f.write("Parameters:\n")
            for key, value in config['global_vars'].items():
                f.write(f"  {key}: {value}\n")
            f.write("\nTraining settings:\n")
            for key, value in config['training'].items():
                f.write(f"  {key}: {value}\n")
            if 'pde_params' in config:
                f.write("\nPDE parameters:\n")
                for key, value in config['pde_params'].items():
                    f.write(f"  {key}: {value}\n")
            f.write("\n\n")

    results = {}
    original_globals = {}
    for var in ['E', 'nu', 'L', 'W', 'H', 'scale_factor', 'total_time',
                'mu', 'lambda_', 'kappa', 'mu_effective', 'lambda_effective']:
        if var in globals():
            original_globals[var] = globals()[var]

    for i, config in enumerate(configurations):
        config_dir = os.path.join(master_dir, config['name'])
        os.makedirs(config_dir, exist_ok=True)
        os.makedirs(os.path.join(config_dir, "model"), exist_ok=True)
        os.makedirs(os.path.join(config_dir, "results", "training_progress"), exist_ok=True)
        os.makedirs(os.path.join(config_dir, "results", "3d_visualizations"), exist_ok=True)
        os.makedirs(os.path.join(config_dir, "results", "animation"), exist_ok=True)

        print("\n" + "=" * 80)
        print(f"TRAINING CONFIGURATION {i+1}: {config['name']}")
        print(f"Description: {config['description']}")
        print("=" * 80)

        update_global_variables(config['global_vars'])
        if 'pde_params' in config:
            update_pde_params(config['pde_params'])

        original_dir = os.getcwd()
        os.chdir(os.path.join(config_dir))

        if base_model is not None and i == 0:
            model = base_model
        else:
            model = create_enhanced_model()

        start_time = time.time()
        force_levels = config['training']['force_levels']
        loss_weights = config['training']['loss_weights']

        trained_model = train_with_curriculum(
            model,
            force_levels=force_levels,
            iterations_per_level=800,
            custom_loss_weights=loss_weights
        )
        training_time = time.time() - start_time
        os.chdir(original_dir)

        generate_visualizations(trained_model, config_dir)
        export_comparison_data(trained_model, config_dir)
        save_config_details(config, config_dir, training_time)

        results[config['name']] = {
            'model': trained_model,
            'training_time': training_time,
            'config': config,
            'output_dir': config_dir
        }
        print(f"Configuration {i+1} completed in {training_time:.2f} seconds")

    for var, value in original_globals.items():
        globals()[var] = value

    generate_comparative_analysis(results, master_dir)
    compress_results(master_dir)

    print("\n" + "=" * 80)
    print("MULTI-CONFIGURATION TRAINING COMPLETED")
    print(f"Results saved to: {master_dir}")
    print("=" * 80)

    return results, master_dir


In [ ]:
def update_global_variables(var_dict):
    print("Updating global variables:")
    for var_name, value in var_dict.items():
        if var_name in globals():
            globals()[var_name] = value
            print(f"  {var_name} = {value}")
        else:
            print(f"  Warning: Variable {var_name} not found in globals")
    if 'E' in var_dict or 'nu' in var_dict:
        global mu, lambda_, kappa, mu_effective, lambda_effective
        E_val = globals().get('E')
        nu_val = globals().get('nu')
        scale_factor = globals().get('scale_factor')
        if E_val is not None and nu_val is not None:
            mu = E_val / (2 * (1 + nu_val))
            lambda_ = E_val * nu_val / ((1 + nu_val) * (1 - 2 * nu_val))
            kappa = E_val / (3 * (1 - 2 * nu_val))
            if scale_factor is not None:
                mu_effective = mu / scale_factor
                lambda_effective = lambda_ / scale_factor
            else:
                mu_effective = mu / globals().get('scale_factor', 5.0)
                lambda_effective = lambda_ / globals().get('scale_factor', 5.0)
            print("Derived parameters updated:")
            print(f"  μ = {mu:.1e} Pa")
            print(f"  λ = {lambda_:.1e} Pa")
            print(f"  μ_effective = {mu_effective:.1e} Pa")
            print(f"  λ_effective = {lambda_effective:.1e} Pa")

def update_pde_params(param_dict):
    print("Updating PDE parameters:")
    for param_name, value in param_dict.items():
        if param_name == 'force_magnitude':
            print(f"  force_magnitude set to {value}")
            print("  Using compensation approach for force magnitude")
            original_force = -5000.0
            force_ratio = value / original_force
            global FORCE_LEVEL
            if 'FORCE_LEVEL' in globals():
                print(f"  Adjusting FORCE_LEVEL from {FORCE_LEVEL} to {FORCE_LEVEL * force_ratio}")
                FORCE_LEVEL *= force_ratio
        else:
            print(f"  {param_name}: {value}")

def generate_visualizations(model, config_dir):
    print("\nGenerating enhanced visualizations...")
    vis_dir = os.path.join(config_dir, "results")
    os.makedirs(vis_dir, exist_ok=True)

    enhanced_visualize_3d_deformation(model, exaggerate_factors=[1.0, 2.0, 5.0, 10.0])
    create_enhanced_animation(model, num_frames=30, exaggerate_factor=5.0)

    try:
        x = np.linspace(0, L, 100)
        y_center = np.ones_like(x) * (W / 2)
        z_center = np.ones_like(x) * (H / 2)
        t_final = np.ones_like(x) * total_time
        centerline_points = np.column_stack([x, y_center, z_center, t_final])
        predictions = model.predict(centerline_points)
        disp_x = predictions[:, 0]
        disp_y = predictions[:, 1]
        disp_z = predictions[:, 2]

        plt.figure(figsize=(12, 6))
        plt.plot(x, disp_x, 'r-', label='X displacement')
        plt.plot(x, disp_y, 'g-', label='Y displacement')
        plt.plot(x, disp_z, 'b-', label='Z displacement')
        plt.xlabel('Position along beam (X)')
        plt.ylabel('Displacement (m)')
        plt.title('Displacement Distribution Along Centerline')
        plt.grid(True)
        plt.legend()
        plt.savefig(os.path.join(vis_dir, "centerline_displacement.png"), dpi=300)
        plt.close()
    except Exception as e:
        print(f"Error generating centerline plot: {e}")

    print("Visualizations completed")

def export_comparison_data(model, config_dir):
    print("\nExporting comparison data...")
    export_dir = os.path.join(config_dir, "comparison_data")
    os.makedirs(export_dir, exist_ok=True)
    try:
        nx, ny, nz = 20, 4, 4
        x = np.linspace(0, L, nx)
        y = np.linspace(0, W, ny)
        z = np.linspace(0, H, nz)
        X, Y, Z = np.meshgrid(x, y, z)
        points = np.column_stack([X.flatten(), Y.flatten(), Z.flatten(), np.ones(X.size) * total_time])
        predictions = model.predict(points)
        df = pd.DataFrame({
            'X': points[:, 0],
            'Y': points[:, 1],
            'Z': points[:, 2],
            'Disp_X': predictions[:, 0],
            'Disp_Y': predictions[:, 1],
            'Disp_Z': predictions[:, 2],
            'Deformed_X': points[:, 0] + predictions[:, 0],
            'Deformed_Y': points[:, 1] + predictions[:, 1],
            'Deformed_Z': points[:, 2] + predictions[:, 2]
        })
        df.to_csv(os.path.join(export_dir, "pinn_deformation_data.csv"), index=False)
        print(f"Exported data to {export_dir}")
    except Exception as e:
        print(f"Error exporting data: {e}")

def save_config_details(config, config_dir, training_time):
    print("\nSaving configuration details...")
    with open(os.path.join(config_dir, "config_summary.txt"), "w") as f:
        f.write(f"Configuration: {config['name']}\n")
        f.write("=" * len(config['name']) + "\n\n")
        f.write(f"Description: {config['description']}\n\n")
        f.write("Parameters:\n")
        for key, value in config['global_vars'].items():
            f.write(f"  {key}: {value}\n")
        f.write("\nTraining settings:\n")
        for key, value in config['training'].items():
            f.write(f"  {key}: {value}\n")
        if 'pde_params' in config:
            f.write("\nPDE parameters:\n")
            for key, value in config['pde_params'].items():
                f.write(f"  {key}: {value}\n")
        f.write(f"\nTraining time: {training_time:.2f} seconds\n")
    print(f"Configuration details saved to {config_dir}")


In [ ]:
def main(mode="single"):
    """
    Main function to run training and generate visualizations.

    Args:
        mode: "single" for one configuration, "multi" for multiple configurations.
    """
    os.makedirs("model", exist_ok=True)
    os.makedirs("results", exist_ok=True)

    if mode == "single":
        with open("model/config.txt", "w") as f:
            f.write("PINN Beam Model Configuration\n")
            f.write("============================\n\n")
            f.write("Physical Parameters:\n")
            f.write(f"  Geometry: L={L} m, W={W} m, H={H} m\n")
            f.write(f"  Material: E={E:.1e} Pa, ν={nu:.3f}\n")
            f.write(f"  Density: {rho} kg/m³\n")
            f.write(f"  Maximum force: {3000:.1f} N\n")
            f.write(f"  Gravity: {gravity_vector}\n\n")
            f.write("Derived Parameters:\n")
            f.write(f"  Shear modulus (μ): {mu:.1e} Pa\n")
            f.write(f"  Lamé parameter (λ): {lambda_:.1e} Pa\n")
            f.write(f"  Bulk modulus (κ): {kappa:.1e} Pa\n")

        print("\n" + "="*80)
        print("PINN Beam Model Training")
        print("="*80)
        print("\nPhysical Parameters:")
        print(f"  Geometry: L={L} m, W={W} m, H={H} m")
        print(f"  Material: E={E:.1e} Pa, ν={nu:.3f}")
        print(f"  Density: {rho} kg/m³")
        print(f"  Maximum force: {3000:.1f} N")
        print(f"  Gravity: {gravity_vector}")
        print("\nDerived Parameters:")
        print(f"  Shear modulus (μ): {mu:.1e} Pa")
        print(f"  Lamé parameter (λ): {lambda_:.1e} Pa")
        print(f"  Bulk modulus (κ): {kappa:.1e} Pa")
        print("="*80 + "\n")

        print("Creating improved PINN model...")
        model = create_enhanced_model()
        print("Starting training with curriculum learning...")
        trained_model = train_with_curriculum(
            model,
            force_levels=[0.1, 0.25, 0.5, 0.75, 1.0, 1.2],
            iterations_per_level=1000
        )
        print("\nGenerating final 3D visualizations...")
        enhanced_visualize_3d_deformation(trained_model, exaggerate_factors=[1.0, 3.0, 5.0, 10.0])
        print("\nCreating deformation animation...")
        create_enhanced_animation(trained_model, num_frames=40, exaggerate_factor=5.0)
        trained_model.save("trained_hyperelastic_beam.dat")

        print("\nTraining and visualization completed successfully!")
        print("Physical parameters used in this model:")
        print(f"  Geometry: L={L} m, W={W} m, H={H} m")
        print(f"  Material: E={E:.1e} Pa, ν={nu:.3f}")
        print(f"  Density: {rho} kg/m³")
        print(f"  Maximum force: {FORCE_LEVEL*3000:.1f} N")
        return trained_model

    elif mode == "multi":
        print("\n" + "="*80)
        print("PINN Beam Model Multi-Configuration Training")
        print("="*80)
        results, master_dir = multi_config_training()
        print("\nMulti-configuration training completed successfully!")
        print(f"Results saved to: {master_dir}")
        return results, master_dir

    else:
        print(f"Training mode '{mode}' not recognized. Valid options: 'single', 'multi'")
        return None

if __name__ == "__main__":
    os.makedirs("model", exist_ok=True)
    os.makedirs("results", exist_ok=True)
    os.environ["DDE_BACKEND"] = "pytorch"
    print("PINN Hyperelastic Beam Simulation")
    print("=" * 80)
    print(f"Using DeepXDE backend: {os.environ['DDE_BACKEND']}")
    dde.backend.set_default_backend("pytorch")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    print(f"GPU available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU name: {torch.cuda.get_device_name(0)}")

    print("\nSelect an option:")
    print("1. Train a single configuration")
    print("2. Train multiple configurations")
    print("3. Compare with SOFA results")

    choice = input("Enter your choice (1-3): ")

    if choice == "1":
        print("\nStarting single configuration training...")
        trained_model = main(mode="single")
        print("Training complete!")
    elif choice == "2":
        print("\nStarting multi-configuration training...")
        results, master_dir = main(mode="multi")
        print("Multi-configuration training complete!")
        print(f"Results saved to: {master_dir}")
        download = input("Do you want to download the results? (y/n): ")
        if download.lower() == "y":
            descargar_resultados_entrenamiento(master_dir)
    elif choice == "3":
        print("\nComparing with SOFA results...")
        model_path = input("Enter path to trained PINN model (.dat file): ")
        sofa_vtk_path = input("Enter path to SOFA VTK/VTU file: ")
        if not os.path.exists(model_path):
            print(f"Error: Model file {model_path} not found.")
        elif not os.path.exists(sofa_vtk_path):
            print(f"Error: SOFA VTK file {sofa_vtk_path} not found.")
        else:
            try:
                model = dde.Model.load(model_path)
                print(f"Model loaded from {model_path}")
                metrics = run_sofa_pinn_comparison(
                    model=model,
                    sofa_vtk_path=sofa_vtk_path,
                    total_time=total_time,
                    output_dir="sofa_pinn_comparison"
                )
                print("Comparison complete!")
            except Exception as e:
                print(f"Error loading model or running comparison: {e}")
    else:
        print("Invalid choice. Please run the script again and select a valid option (1-3).")
